Purpose: Classification, Clustering. <br>
Source of download:
https://archive.ics.uci.edu/ml/datasets/diabetes+130-us+hospitals+for+years+1999-2008# <br>
Description of data features:
https://www.hindawi.com/journals/bmri/2014/781670/tab1/

Import libaries:

In [56]:
import numpy as np
import pandas as pd

In [57]:
df = pd.read_csv("diabetic_data.csv")
df = pd.DataFrame(df)
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


Drop columns that we cannot use or do not understand:

In [58]:
df = df.drop(columns = ['encounter_id', 'patient_nbr', "admission_type_id", "discharge_disposition_id",
                        'admission_source_id',
                        "payer_code", "medical_specialty",
                        'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
                        'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
                        'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
                        'tolazamide', 'examide', 'citoglipton',
                        'glyburide-metformin', 'glipizide-metformin',
                        'glimepiride-pioglitazone', 'metformin-rosiglitazone', 
                        'metformin-pioglitazone', 'number_outpatient',
                        'diag_1','diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult'])

We dropped all but one medication name (insulin) because we want to experiment with one only.

Names of columns:

In [59]:
df.columns

Index(['race', 'gender', 'age', 'weight', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_emergency', 'number_inpatient', 'number_diagnoses', 'insulin',
       'change', 'diabetesMed', 'readmitted'],
      dtype='object')

Check unique values to determine how to change values to meaningful ones:

In [60]:
for i in range(len(df.columns)):
    col_name = df.columns[i]
    print(col_name)
    print(df[col_name].unique())

race
['Caucasian' 'AfricanAmerican' '?' 'Other' 'Asian' 'Hispanic']
gender
['Female' 'Male' 'Unknown/Invalid']
age
['[0-10)' '[10-20)' '[20-30)' '[30-40)' '[40-50)' '[50-60)' '[60-70)'
 '[70-80)' '[80-90)' '[90-100)']
weight
['?' '[75-100)' '[50-75)' '[0-25)' '[100-125)' '[25-50)' '[125-150)'
 '[175-200)' '[150-175)' '>200']
time_in_hospital
[ 1  3  2  4  5 13 12  9  7 10  6 11  8 14]
num_lab_procedures
[ 41  59  11  44  51  31  70  73  68  33  47  62  60  55  49  75  45  29
  35  42  66  36  19  64  25  53  52  87  27  37  46  28  48  72  10   2
  65  67  40  54  58  57  43  32  83  34  39  69  38  56  22  96  78  61
  88  50   1  18  82   9  63  24  71  77  81  76  90  93   3 103  13  80
  85  16  15  12  30  23  17  21  79  26   5  95  97  84  14  74 105  86
  98  20   6  94   8 102 100   7  89  91  92   4 101  99 114 113 111 129
 107 108 106 104 109 120 132 121 126 118]
num_procedures
[0 5 1 6 2 3 4]
num_medications
[ 1 18 13 16  8 21 12 28 17 11 15 31  2 23 19  7 20 14 10 22  9 27

We see that two variables, weight, race and gender have same observations with unknown values. Weight and race have '?', although Weight could be converted to and ordinal categorical variable. So let us replace them with something meaningful.

Now change some categorical variable values from strings to ordinal numeric ones:

In [61]:
df['change'] = df['change'].replace(['No', 'Ch'], [0,1])
df['diabetesMed'] = df['diabetesMed'].replace(['No', 'Yes'], [0,1])
df['insulin'] = df['insulin'].replace(['No', 'Up', 'Steady', 'Down'], [0,3,2,1])
df['readmitted'] = df['readmitted'].replace(['NO', '>30', '<30'], [0,2,1])
df['age'] = df['age'].replace(['[0-10)', '[10-20)', '[20-30)', '[30-40)', '[40-50)', 
                               '[50-60)', '[60-70)', '[70-80)', '[80-90)', '[90-100)'], 
                              [0,1,2,3,4,5,6,7,8,9])
df.head(10)

,race,gender,age,weight,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_emergency,number_inpatient,number_diagnoses,insulin,change,diabetesMed,readmitted
0,Caucasian,Female,0,?,1,41,0,1,0,0,1,0,0,0,0
1,Caucasian,Female,1,?,3,59,0,18,0,0,9,3,1,1,2
2,AfricanAmerican,Female,2,?,2,11,5,13,0,1,6,0,0,1,0
3,Caucasian,Male,3,?,2,44,1,16,0,0,7,3,1,1,0
4,Caucasian,Male,4,?,1,51,0,8,0,0,5,2,1,1,0
5,Caucasian,Male,5,?,3,31,6,16,0,0,9,2,0,1,2
6,Caucasian,Male,6,?,4,70,1,21,0,0,7,2,1,1,0
7,Caucasian,Male,7,?,5,73,0,12,0,0,8,0,0,1,2
8,Caucasian,Female,8,?,13,68,2,28,0,0,8,2,1,1,0
9,Caucasian,Female,9,?,12,33,3,18,0,0,8,2,1,1,0


For each of the three suspect variables, check number of observations with unknown values so we can decide if it is small enough for us to remove the observations without having a massive impact on the overall distribution of the dataset.

In [62]:
print([df['race'].value_counts()['?'],
      df['gender'].value_counts()['Unknown/Invalid'],
      df['weight'].value_counts()['?']])

[2273, 3, 98569]


Only about 2 percent of values are perhaps worth removing. This is low enough, so we'll remove all values where this occurs. And nearly all of the values have unknown weight, so we can remove the weight variable. Then we only have 15 columns.

In [63]:
df = df[df["race"] != '?']
df = df[df["gender"] != 'Unknown/Invalid']
df = df.drop(columns= 'weight')
df

,race,gender,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_emergency,number_inpatient,number_diagnoses,insulin,change,diabetesMed,readmitted
0,Caucasian,Female,0,1,41,0,1,0,0,1,0,0,0,0
1,Caucasian,Female,1,3,59,0,18,0,0,9,3,1,1,2
2,AfricanAmerican,Female,2,2,11,5,13,0,1,6,0,0,1,0
3,Caucasian,Male,3,2,44,1,16,0,0,7,3,1,1,0
4,Caucasian,Male,4,1,51,0,8,0,0,5,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,AfricanAmerican,Male,7,3,51,0,16,0,0,9,1,1,1,2
101762,AfricanAmerican,Female,8,5,33,3,18,0,1,9,2,0,1,0
101763,Caucasian,Male,7,1,53,0,9,0,0,13,1,1,1,0
101764,Caucasian,Female,8,10,45,2,21,0,1,9,3,1,1,0


Now we have 15 variables - abundant enough for us to make a diverse range of analyses with it. We have 13 numeric variables, all of which are ordinal and 2 categorical ones which aren't ordinal.

Next, encode the race and gender variables into dummy variables using pandas' get_dummies.

In [64]:
df = pd.get_dummies(df, prefix=['race', 'gender'])

Export this dataframe to csv file for use. We will develop ML models in another notebook using it.

In [65]:
df.to_csv('diabetes.csv') 

Now that we have wrangled with and cleaned the data, we can move on to modeling the data using supervised learning - regression and classification.